In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates

In [32]:
#import data
harvest_perrault = pd.read_csv('harvester_perrault.csv')
harvest_pilot = pd.read_csv('harvest_pilot.csv')

In [33]:
harvest_pilot['productive_minutes'] = harvest_pilot['strings'] / harvest_pilot['vpm']

In [34]:
pilot = harvest_pilot.groupby('variety')['productive_minutes'].sum().reset_index()

In [46]:
pilot  

,variety,productive_minutes,order
0,Cascade,555.317077,0
5,Simcoe,2844.198612,1
1,Citra,2052.972642,2
4,Mosaic,448.106181,3
3,HBC 682,506.610797,4
2,HBC 586,2286.922384,5


In [35]:
pilot['productive_minutes'].sum()

9144.12769362297

In [36]:
pilot['variety'].unique()

array(['Cascade', 'Citra', 'HBC 586', 'HBC 682', 'Mosaic', 'Simcoe'],
      dtype=object)

In [37]:
from datetime import datetime, timedelta
import pandas as pd

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(date):
    return date.weekday() < 6

# Function to find the next working day starting from current_datetime
def next_working_day(current_datetime):
    next_day = current_datetime + timedelta(days=1)
    while not is_working_day(next_day):
        next_day += timedelta(days=1)
    return next_day

# Function to calculate effective shift time and break time in a shift
def calculate_shift_time_with_breaks(shift_length=timedelta(minutes=600), break_duration=10, lunch_duration=30):
    work_period = timedelta(minutes=240)
    num_breaks_per_shift = shift_length // work_period
    total_break_time_per_shift = num_breaks_per_shift * timedelta(minutes=break_duration)
    effective_shift_time = shift_length - timedelta(minutes=lunch_duration)
    return effective_shift_time, total_break_time_per_shift.total_seconds() / 60  # Convert break time to minutes

# Function to create the shift schedule for a single shift starting at 6:00 AM
def create_single_shift_schedule_with_breaks(data, start_datetime, variety_shift_lengths, break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    # Convert variety_shift_lengths DataFrame to a dictionary for faster lookups
    shift_length_dict = variety_shift_lengths.set_index('variety')['length'].to_dict()

    while not data.empty:
        for i, row in data.iterrows():
            variety = row['variety']
            remaining_minutes = row['productive_minutes']

            # Get the shift length for this variety
            shift_length = shift_length_dict.get(variety, timedelta(minutes=600))  # Default to 600 minutes if not found

            while remaining_minutes > 0:
                effective_shift_time, total_break_time_per_shift = calculate_shift_time_with_breaks(
                    shift_length=shift_length, break_duration=break_duration, lunch_duration=lunch_duration)

                # Skip non-working days
                if not is_working_day(current_datetime):
                    current_datetime = next_working_day(current_datetime)

                # Set the shift start time to 6:00 AM
                shift_start_time = current_datetime.replace(hour=6, minute=0)
                shift_end_time = shift_start_time + shift_length
                actual_shift_time = min(remaining_minutes, effective_shift_time.total_seconds() / 60)
                remaining_minutes -= actual_shift_time

                schedule.append({
                    'Variety': variety,
                    'ShiftStart': shift_start_time,
                    'ShiftEnd': shift_end_time,
                    'ShiftTime': shift_length.total_seconds() / 60,  # Convert to minutes
                    'EffectiveShiftTime': actual_shift_time,
                    'RemainingMinutes': remaining_minutes,
                    'BreakTime': total_break_time_per_shift,
                    'LunchBreak': lunch_duration
                })

                # Move to the next working day for the next shift
                current_datetime = next_working_day(shift_end_time)

            # Deduct the leftover from the next variety's remaining minutes
            if remaining_minutes < (effective_shift_time.total_seconds() / 60):  # Convert effective_shift_time to minutes
                leftover = (effective_shift_time.total_seconds() / 60) - remaining_minutes
                remaining_minutes = 0  # Set current variety's remaining minutes to 0

                # Deduct leftover from the next variety's remaining minutes
                if i + 1 < len(data):
                    next_variety_minutes = data.at[i + 1, 'productive_minutes']
                    if next_variety_minutes >= leftover:
                        data.at[i + 1, 'productive_minutes'] -= leftover
                    else:
                        # If leftover exceeds next variety's minutes, set next variety's minutes to 0
                        data.at[i + 1, 'productive_minutes'] = 0
                        leftover -= next_variety_minutes

            # If remaining minutes for this variety are zero, remove it from the DataFrame
            if remaining_minutes <= 0:
                data = data.drop(i).reset_index(drop=True)
                break  # Move to the next variety in the next iteration

    return pd.DataFrame(schedule)

# Example usage
start_datetime = datetime(2024, 9, 1, 6, 0)  # Start time is fixed at 6:00 AM

pilot = pd.DataFrame({
    'variety': ['Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682', 'HBC 586'],
    'productive_minutes': [555.317077, 2844.198612, 2052.972642, 448.106181, 506.610797, 2286.922384]  # Example data
})


# Example variety_shift_lengths DataFrame (single shift length used)
variety_shift_lengths = pd.DataFrame({
    'variety': ['Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682','HBC 586'],
    'length': [timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480) ] 
})

# Define the order in which varieties need to be picked
picking_order = [
    'Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682', 'HBC 586'
]

# Sort the DataFrame based on the picking order
pilot['order'] = pilot['variety'].apply(lambda x: picking_order.index(x))
pilot = pilot.sort_values(by='order', ascending=True)

# Assuming 'pilot' is your DataFrame with the appropriate structure
shift_schedule = create_single_shift_schedule_with_breaks(
    data=pilot,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths  # Pass the DataFrame with the shift lengths
)

# Output the schedule
print(shift_schedule)


    Variety          ShiftStart            ShiftEnd  ShiftTime  \
0   Cascade 2024-09-02 06:00:00 2024-09-02 14:00:00      480.0   
1   Cascade 2024-09-03 06:00:00 2024-09-03 14:00:00      480.0   
2    Simcoe 2024-09-04 06:00:00 2024-09-04 14:00:00      480.0   
3    Simcoe 2024-09-05 06:00:00 2024-09-05 14:00:00      480.0   
4    Simcoe 2024-09-06 06:00:00 2024-09-06 14:00:00      480.0   
5    Simcoe 2024-09-07 06:00:00 2024-09-07 14:00:00      480.0   
6    Simcoe 2024-09-09 06:00:00 2024-09-09 14:00:00      480.0   
7    Simcoe 2024-09-10 06:00:00 2024-09-10 14:00:00      480.0   
8    Simcoe 2024-09-11 06:00:00 2024-09-11 14:00:00      480.0   
9     Citra 2024-09-12 06:00:00 2024-09-12 14:00:00      480.0   
10    Citra 2024-09-13 06:00:00 2024-09-13 14:00:00      480.0   
11    Citra 2024-09-14 06:00:00 2024-09-14 14:00:00      480.0   
12    Citra 2024-09-16 06:00:00 2024-09-16 14:00:00      480.0   
13  HBC 682 2024-09-17 06:00:00 2024-09-17 14:00:00      480.0   
14  HBC 58

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from datetime import datetime, timedelta
import pandas as pd
import config_pilot 

def run_script():
    # Get user inputs
    start_date = simpledialog.askstring("Input", "Enter the start date (YYYY-MM-DD):")
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d").replace(hour=6, minute=0)

    picking_order_input = simpledialog.askstring("Input", "Enter the picking order, separated by commas (e.g., Cascade,Simcoe,Citra,Mosaic,HBC 682,HBC 586):")
    picking_order = [x.strip() for x in picking_order_input.split(',')]

    # Collect shift lengths for each variety
    variety_shift_lengths = []
    for variety in picking_order:
        shift_length_minutes = simpledialog.askinteger("Input", f"Enter the shift length in minutes for {variety}:")
        variety_shift_lengths.append({'variety': variety, 'length': timedelta(minutes=shift_length_minutes)})

    # Convert to DataFrame
    variety_shift_lengths_df = pd.DataFrame(variety_shift_lengths)

    # Display the collected inputs
    print(f"Start Date: {start_datetime}")
    print(f"Variety Shift Lengths:\n{variety_shift_lengths_df}")

    # Continue with the rest of the script logic...

root = tk.Tk()
root.withdraw()  # Hide the main window
run_script()

import configparser
from datetime import datetime, timedelta
import pandas as pd

# Read configuration from a file
config = configparser.ConfigParser()
config.read('config.ini')

start_date = config['DEFAULT']['StartDate']
start_datetime = datetime.strptime(start_date, "%Y-%m-%d").replace(hour=6, minute=0)

picking_order = config['DEFAULT']['PickingOrder'].split(',')
shift_lengths = [int(x.strip()) for x in config['DEFAULT']['ShiftLengths'].split(',')]

# Create a DataFrame with variety and corresponding shift lengths
variety_shift_lengths_df = pd.DataFrame({
    'variety': [x.strip() for x in picking_order],
    'length': [timedelta(minutes=minutes) for minutes in shift_lengths]
})

# Display the collected inputs
print(f"Start Date: {start_datetime}")
print(f"Variety Shift Lengths:\n{variety_shift_lengths_df}")

from datetime import datetime, timedelta
import pandas as pd

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(date):
    return date.weekday() < 6

# Function to find the next working day starting from current_datetime
def next_working_day(current_datetime):
    next_day = current_datetime + timedelta(days=1)
    while not is_working_day(next_day):
        next_day += timedelta(days=1)
    return next_day

# Function to calculate effective shift time and break time in a shift
def calculate_shift_time_with_breaks(shift_length=timedelta(minutes=600), break_duration=10, lunch_duration=30):
    work_period = timedelta(minutes=240)
    num_breaks_per_shift = shift_length // work_period
    total_break_time_per_shift = num_breaks_per_shift * timedelta(minutes=break_duration)
    effective_shift_time = shift_length - timedelta(minutes=lunch_duration)
    return effective_shift_time, total_break_time_per_shift.total_seconds() / 60  # Convert break time to minutes

# Function to create the shift schedule for a single shift starting at 6:00 AM
def create_single_shift_schedule_with_breaks(data, start_datetime, variety_shift_lengths, break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    # Convert variety_shift_lengths DataFrame to a dictionary for faster lookups
    shift_length_dict = variety_shift_lengths.set_index('variety')['length'].to_dict()

    while not data.empty:
        for i, row in data.iterrows():
            variety = row['variety']
            remaining_minutes = row['productive_minutes']

            # Get the shift length for this variety
            shift_length = shift_length_dict.get(variety, timedelta(minutes=600))  # Default to 600 minutes if not found

            while remaining_minutes > 0:
                effective_shift_time, total_break_time_per_shift = calculate_shift_time_with_breaks(
                    shift_length=shift_length, break_duration=break_duration, lunch_duration=lunch_duration)

                # Skip non-working days
                if not is_working_day(current_datetime):
                    current_datetime = next_working_day(current_datetime)

                # Set the shift start time to 6:00 AM
                shift_start_time = current_datetime.replace(hour=6, minute=0)
                shift_end_time = shift_start_time + shift_length
                actual_shift_time = min(remaining_minutes, effective_shift_time.total_seconds() / 60)
                remaining_minutes -= actual_shift_time

                schedule.append({
                    'Variety': variety,
                    'ShiftStart': shift_start_time,
                    'ShiftEnd': shift_end_time,
                    'ShiftTime': shift_length.total_seconds() / 60,  # Convert to minutes
                    'EffectiveShiftTime': actual_shift_time,
                    'RemainingMinutes': remaining_minutes,
                    'BreakTime': total_break_time_per_shift,
                    'LunchBreak': lunch_duration
                })

                # Move to the next working day for the next shift
                current_datetime = next_working_day(shift_end_time)

            # Deduct the leftover from the next variety's remaining minutes
            if remaining_minutes < (effective_shift_time.total_seconds() / 60):  # Convert effective_shift_time to minutes
                leftover = (effective_shift_time.total_seconds() / 60) - remaining_minutes
                remaining_minutes = 0  # Set current variety's remaining minutes to 0

                # Deduct leftover from the next variety's remaining minutes
                if i + 1 < len(data):
                    next_variety_minutes = data.at[i + 1, 'productive_minutes']
                    if next_variety_minutes >= leftover:
                        data.at[i + 1, 'productive_minutes'] -= leftover
                    else:
                        # If leftover exceeds next variety's minutes, set next variety's minutes to 0
                        data.at[i + 1, 'productive_minutes'] = 0
                        leftover -= next_variety_minutes

            # If remaining minutes for this variety are zero, remove it from the DataFrame
            if remaining_minutes <= 0:
                data = data.drop(i).reset_index(drop=True)
                break  # Move to the next variety in the next iteration

    return pd.DataFrame(schedule)

# Example usage
start_datetime = datetime(2024, 9, 1, 6, 0)  # Start time is fixed at 6:00 AM


# Example variety_shift_lengths DataFrame (single shift length used)
variety_shift_lengths = pd.DataFrame({
    'variety': ['Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682','HBC 586'],
    'length': [timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480) ]
})

# Define the order in which varieties need to be picked
picking_order = [
    'Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682', 'HBC 586'
]

# Sort the DataFrame based on the picking order
pilot['order'] = pilot['variety'].apply(lambda x: picking_order.index(x))
pilot = pilot.sort_values(by='order', ascending=True)

# Assuming 'pilot' is your DataFrame with the appropriate structure
shift_schedule = create_single_shift_schedule_with_breaks(
    data=pilot,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths  # Pass the DataFrame with the shift lengths
)

# Output the schedule
print(shift_schedule)


In [56]:
import tkinter as tk
from tkinter import simpledialog
from datetime import datetime, timedelta
import pandas as pd
import configparser
import config_pilot

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(date):
    return date.weekday() < 6

# Function to find the next working day starting from current_datetime
def next_working_day(current_datetime):
    next_day = current_datetime + timedelta(days=1)
    while not is_working_day(next_day):
        next_day += timedelta(days=1)
    return next_day

# Function to calculate effective shift time and break time in a shift
def calculate_shift_time_with_breaks(shift_length=timedelta(minutes=600), break_duration=10, lunch_duration=30):
    work_period = timedelta(minutes=240)
    num_breaks_per_shift = shift_length // work_period
    total_break_time_per_shift = num_breaks_per_shift * timedelta(minutes=break_duration)
    effective_shift_time = shift_length - timedelta(minutes=lunch_duration)
    return effective_shift_time, total_break_time_per_shift.total_seconds() / 60  # Convert break time to minutes

# Function to create the shift schedule for a single shift starting at 6:00 AM
def create_single_shift_schedule_with_breaks(data, start_datetime, variety_shift_lengths, break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    # Convert variety_shift_lengths DataFrame to a dictionary for faster lookups
    shift_length_dict = variety_shift_lengths.set_index('variety')['length'].to_dict()

    while not data.empty:
        for i, row in data.iterrows():
            variety = row['variety']
            remaining_minutes = row['productive_minutes']

            # Get the shift length for this variety
            shift_length = shift_length_dict.get(variety, timedelta(minutes=600))  # Default to 600 minutes if not found

            while remaining_minutes > 0:
                effective_shift_time, total_break_time_per_shift = calculate_shift_time_with_breaks(
                    shift_length=shift_length, break_duration=break_duration, lunch_duration=lunch_duration)

                # Skip non-working days
                if not is_working_day(current_datetime):
                    current_datetime = next_working_day(current_datetime)

                # Set the shift start time to 6:00 AM
                shift_start_time = current_datetime.replace(hour=6, minute=0)
                shift_end_time = shift_start_time + shift_length
                actual_shift_time = min(remaining_minutes, effective_shift_time.total_seconds() / 60)
                remaining_minutes -= actual_shift_time

                schedule.append({
                    'Variety': variety,
                    'ShiftStart': shift_start_time,
                    'ShiftEnd': shift_end_time,
                    'ShiftTime': shift_length.total_seconds() / 60,  # Convert to minutes
                    'EffectiveShiftTime': actual_shift_time,
                    'RemainingMinutes': remaining_minutes,
                    'BreakTime': total_break_time_per_shift,
                    'LunchBreak': lunch_duration
                })

                # Move to the next working day for the next shift
                current_datetime = next_working_day(shift_end_time)

            # Deduct the leftover from the next variety's remaining minutes
            if remaining_minutes < (effective_shift_time.total_seconds() / 60):  # Convert effective_shift_time to minutes
                leftover = (effective_shift_time.total_seconds() / 60) - remaining_minutes
                remaining_minutes = 0  # Set current variety's remaining minutes to 0

                # Deduct leftover from the next variety's remaining minutes
                if i + 1 < len(data):
                    next_variety_minutes = data.at[i + 1, 'productive_minutes']
                    if next_variety_minutes >= leftover:
                        data.at[i + 1, 'productive_minutes'] -= leftover
                    else:
                        # If leftover exceeds next variety's minutes, set next variety's minutes to 0
                        data.at[i + 1, 'productive_minutes'] = 0
                        leftover -= next_variety_minutes

            # If remaining minutes for this variety are zero, remove it from the DataFrame
            if remaining_minutes <= 0:
                data = data.drop(i).reset_index(drop=True)
                break  # Move to the next variety in the next iteration

    return pd.DataFrame(schedule)

# Function to run the script using inputs from a GUI
def run_script_with_gui():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Get user inputs
    start_date = simpledialog.askstring("Input", "Enter the start date (YYYY-MM-DD):")
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d").replace(hour=6, minute=0)

    picking_order_input = simpledialog.askstring('Input', "Enter the picking order, separated by commas (e.g., Cascade,Simcoe,Citra,Mosaic,HBC 682,HBC 586):")
    picking_order = [x.strip() for x in picking_order_input.split(',')]

    # Collect shift lengths for each variety
    variety_shift_lengths = []
    for variety in picking_order:
        shift_length_minutes = simpledialog.askinteger("Input", f"Enter the shift length in minutes for {variety}:")
        variety_shift_lengths.append({'variety': variety, 'length': timedelta(minutes=shift_length_minutes)})

    # Convert to DataFrame
    variety_shift_lengths_df = pd.DataFrame(variety_shift_lengths)

    # Sort the DataFrame based on the picking order
    pilot['order'] = pilot['variety'].apply(lambda x: picking_order.index(x))
    pilot_sorted = pilot.sort_values(by='order', ascending=True)

    # Create the shift schedule
    shift_schedule = create_single_shift_schedule_with_breaks(
        data=pilot_sorted,
        start_datetime=start_datetime,
        variety_shift_lengths=variety_shift_lengths_df  # Pass the DataFrame with the shift lengths
    )

    # Output the schedule
    print(shift_schedule)

# Function to run the script using inputs from a configuration file
def run_script_with_config():
    config = configparser.ConfigParser()
    config.read('config.ini')

    start_date = config['DEFAULT']['StartDate']
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d").replace(hour=6, minute=0)

    picking_order = config['DEFAULT']['PickingOrder'].split(',')
    shift_lengths = [int(x.strip()) for x in config['DEFAULT']['ShiftLength'].split(',')]

    # Create a DataFrame with variety and corresponding shift lengths
    variety_shift_lengths_df = pd.DataFrame({
        'variety': [x.strip() for x in picking_order],
        'length': [timedelta(minutes=minutes) for minutes in shift_lengths]
    })

    # Sort the DataFrame based on the picking order
    pilot['order'] = pilot['variety'].apply(lambda x: picking_order.index(x))
    pilot_sorted = pilot.sort_values(by='order', ascending=True)

    # Create the shift schedule
    shift_schedule = create_single_shift_schedule_with_breaks(
        data=pilot_sorted,
        start_datetime=start_datetime,
        variety_shift_lengths=variety_shift_lengths_df  # Pass the DataFrame with the shift lengths
    )

    # Output the schedule
    print(shift_schedule)

# Example pilot DataFrame with sample data
pilot = pd.DataFrame({
    'variety': ['Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682', 'HBC 586'],
    'productive_minutes': [555.317077, 2844.198612, 2052.972642, 448.106181, 506.610797, 2286.922384]  # Example data
})

# Example usage
# You can choose how to run the script by uncommenting one of the following lines:

# run_script_with_gui()  # Uncomment this to use the GUI for inputs
run_script_with_config()  # Uncomment this to use the configuration file for inputs


KeyError: 'StartDate'

In [47]:
pilot 

,variety,productive_minutes,order
0,Cascade,555.317077,0
5,Simcoe,2844.198612,1
1,Citra,2052.972642,2
4,Mosaic,448.106181,3
3,HBC 682,506.610797,4
2,HBC 586,2286.922384,5


In [59]:
import configparser
from datetime import datetime, timedelta
import pandas as pd
import config_pilot


# Read configuration from a file
config = configparser.ConfigParser()
config.read('config_pilot.py')

# Correct key usage in config
start_date = config['DEFAULT']['StartDate']  # Ensure this matches your config.ini key
start_datetime = datetime.strptime(start_date, "%Y-%m-%d").replace(hour=6, minute=0)

picking_order = [x.strip() for x in config['DEFAULT']['PickingOrder'].split(',')]  # Strip any extra spaces
shift_lengths = [int(x.strip()) for x in config['DEFAULT']['ShiftLengths'].split(',')]

# Create a DataFrame with variety and corresponding shift lengths
variety_shift_lengths_df = pd.DataFrame({
    'variety': [x.strip() for x in picking_order],
    'length': [timedelta(minutes=minutes) for minutes in shift_lengths]
})

# Example pilot DataFrame with sample data
pilot = pd.DataFrame({
    'variety': ['Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682', 'HBC 586'],
    'productive_minutes': [555.317077, 2844.198612, 2052.972642, 448.106181, 506.610797, 2286.922384]  # Example data
})

# Check if all varieties in pilot are in the picking_order
missing_varieties = [variety for variety in pilot['variety'] if variety not in picking_order]
if missing_varieties:
    print(f"Warning: The following varieties are missing in picking_order: {missing_varieties}")

# Sort the DataFrame based on the picking order
pilot['order'] = pilot['variety'].apply(lambda x: picking_order.index(x.strip()))  # Ensure no extra spaces
pilot_sorted = pilot.sort_values(by='order', ascending=True)

# Create the shift schedule
shift_schedule = create_single_shift_schedule_with_breaks(
    data=pilot_sorted,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths_df  # Pass the DataFrame with the shift lengths
)

# Output the schedule
print(shift_schedule)



    Variety          ShiftStart            ShiftEnd  ShiftTime  \
0   Cascade 2024-09-02 06:00:00 2024-09-02 14:00:00      480.0   
1   Cascade 2024-09-03 06:00:00 2024-09-03 14:00:00      480.0   
2    Simcoe 2024-09-04 06:00:00 2024-09-04 14:00:00      480.0   
3    Simcoe 2024-09-05 06:00:00 2024-09-05 14:00:00      480.0   
4    Simcoe 2024-09-06 06:00:00 2024-09-06 14:00:00      480.0   
5    Simcoe 2024-09-07 06:00:00 2024-09-07 14:00:00      480.0   
6    Simcoe 2024-09-09 06:00:00 2024-09-09 14:00:00      480.0   
7    Simcoe 2024-09-10 06:00:00 2024-09-10 14:00:00      480.0   
8     Citra 2024-09-11 06:00:00 2024-09-11 14:00:00      480.0   
9     Citra 2024-09-12 06:00:00 2024-09-12 14:00:00      480.0   
10    Citra 2024-09-13 06:00:00 2024-09-13 14:00:00      480.0   
11    Citra 2024-09-14 06:00:00 2024-09-14 14:00:00      480.0   
12  HBC 682 2024-09-16 06:00:00 2024-09-16 14:00:00      480.0   
13  HBC 586 2024-09-17 06:00:00 2024-09-17 14:00:00      480.0   
14  HBC 58

In [60]:
shift_schedule.to_excel('pilot_schedule.xlsx', index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [63]:
import streamlit as st
from datetime import datetime, timedelta
import pandas as pd

st.title("Shift Scheduler")

start_date = st.date_input("Enter the start date:", datetime(2024, 9, 1))
start_datetime = datetime.combine(start_date, datetime.min.time()).replace(hour=6, minute=0)

picking_order = st.text_input("Enter the picking order, separated by commas:", "Cascade, Simcoe, Citra, Mosaic, HBC 682, HBC 586")
picking_order = [x.strip() for x in picking_order.split(',')]

shift_lengths = st.text_input("Enter the shift lengths for each variety, separated by commas:", "480, 480, 480, 480, 480, 480")
shift_lengths = [int(x.strip()) for x in shift_lengths.split(',')]

# Create a DataFrame with variety and corresponding shift lengths
variety_shift_lengths_df = pd.DataFrame({
    'variety': picking_order,
    'length': [timedelta(minutes=minutes) for minutes in shift_lengths]
})

# Example pilot DataFrame with sample data
pilot = pd.DataFrame({
    'variety': ['Cascade', 'Simcoe', 'Citra', 'Mosaic', 'HBC 682', 'HBC 586'],
    'productive_minutes': [555.317077, 2844.198612, 2052.972642, 448.106181, 506.610797, 2286.922384]  # Example data
})

# Sort the DataFrame based on the picking order
pilot['order'] = pilot['variety'].apply(lambda x: picking_order.index(x))
pilot_sorted = pilot.sort_values(by='order', ascending=True)

# Create the shift schedule
shift_schedule = create_single_shift_schedule_with_breaks(
    data=pilot_sorted,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths_df  # Pass the DataFrame with the shift lengths
)

st.write("Shift Schedule")
st.dataframe(shift_schedule)

if st.button("Export to Excel"):
    shift_schedule.to_excel('shift_schedule.xlsx', index=False)
    st.success("Schedule exported to shift_schedule.xlsx")
